In [1]:
import pandas as pd
from pandas.stats.api import ols
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
%matplotlib inline

###load all the datasets
##meta_data file
filepath = "/Users/eshwarchandrasekharan/Desktop/buzzed_code/"
meta_df = pd.read_csv(filepath + 'facebook/post_metadata_5-18.csv', index_col=0)
meta_df = meta_df.fillna(0)

##all data file
all_df = pd.read_csv(filepath + 'facebook/post_stats_5-18.csv', index_col=0)
all_df = all_df.fillna(0)

In [ ]:
# import random
# sample_id = random.choice(meta_df['buzz_id'])

# temp_comments = all_df[all_df.external_id == sample_id]

# import math
# ainf = math.inf
# t_df

In [188]:
# import random
# sample_id = random.choice(meta_df['buzz_id'])

dataframes = pd.DataFrame()
count = 0

###Note that this is at a buzz_id or link level, and the predictions are on the first posting's performance
for sample_id in meta_df['buzz_id'][:500]:
    print(count, ") Working on: ", sample_id)
    count += 1
    t_df = meta_df[meta_df['buzz_id'] == sample_id]
    output = t_df[['buzz_id', 'title']].drop_duplicates()
    output['start'] = t_df['created_at'].min()

    all_pages = list(meta_df.page.unique())

    def get_cross_posted_pages(x):
        tf = meta_df[meta_df.buzz_id == x].sort_values('created_at', ascending = 1)
        return tf.page.unique()

    for buzz_id in output['buzz_id']:
        output['pages'] = output['buzz_id'].apply(get_cross_posted_pages)


    ##get earliest external_id for each buzz_id
    def get_first_page(x):
        tf = meta_df[meta_df.buzz_id == x].sort_values('created_at', ascending = 1)
        return tf.external_id.values[0]

    output['pages'] = t_df['buzz_id'].apply(get_cross_posted_pages)
    output['external_id'] = output['buzz_id'].apply(get_first_page)

    ###generate the order of cross-promotion:
    def generate_cat_pages(x, y):
        if(y in x):
            return list(x).index(y) + 1
        else:
            return math.inf

    for pag in all_pages:
    #     print("Getting for ", pag)
        output[pag] = output['pages'].apply(generate_cat_pages, y = pag)

    #####GET PERFORMANCE METRICS

    train_offset = 60
    test_offset = 24*60

    all_features = pd.DataFrame()
    all_predictions = pd.DataFrame()

    for external_id in output['external_id']:

        link_df = all_df[all_df['external_id'] == external_id]
        ###get the row for 1st 1 hour
        ###get the row for 1st day (24 hours)
        start = link_df['stats_date'].min()
#         print(start)
        from datetime import datetime
        try:
            dt1 = datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
        except:
            continue
            
        ###within 30 minutes from posting
        import datetime
        ###set the initial offset period of interest - currently anything within the 1st 30 minutes!
        l_train_offset = datetime.timedelta(minutes=train_offset - 1)
        u_train_offset = datetime.timedelta(minutes=train_offset + 1)

        l_test_offset = datetime.timedelta(minutes=test_offset - 1)
        u_test_offset = datetime.timedelta(minutes=test_offset + 60)

        def get_timestamp(x):
            from datetime import datetime
            return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

        link_df['stats_timestamp'] = link_df['stats_date'].apply(get_timestamp)
        features = link_df[(link_df.stats_timestamp > dt1 + l_train_offset) & (link_df.stats_timestamp <= dt1 + u_train_offset)][:1]
        predictions = link_df[(link_df.stats_timestamp > dt1 + l_test_offset) & (link_df.stats_timestamp <= dt1 + u_test_offset)][:1]

        all_features = pd.concat([features,all_features])
        all_predictions = pd.concat([predictions,all_predictions])

        dependent_variables = ['external_id', 'consumptions_by_type__link_clicks', 'consumptions_by_type__other_clicks', 
                               'reactions_like_total', 'stories_by_action_type__comment', 'stories_by_action_type__like', 
                               'stories_by_action_type__share', 'engaged_fan', 'fan_reach',
                               'impressions', 'impressions_fan']
        predictor_variables = ['external_id', 'consumptions_by_type__link_clicks', 'stories_by_action_type__share']
        all_features = all_features[dependent_variables]
        all_predictions = all_predictions[predictor_variables]

        dependent_variables = ["train_" + var if var != "external_id" else var for var in dependent_variables]
        predictor_variables = ["test_" + var if var != "external_id" else var for var in predictor_variables]

        all_features.columns = dependent_variables
        all_predictions.columns = predictor_variables

        output = output.join(all_features.set_index('external_id'), on = 'external_id')
        output = output.join(all_predictions.set_index('external_id'), on = 'external_id')
        
        dataframes = pd.concat([output, dataframes])

dataframes.to_csv('links_cross_posting_info.csv', index = False)

0 ) Working on:  3059339


/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1 ) Working on:  3808033
2 ) Working on:  4056239
3 ) Working on:  4120995
4 ) Working on:  4169766
5 ) Working on:  4183282
6 ) Working on:  4212611
7 ) Working on:  4244759
8 ) Working on:  4297018
9 ) Working on:  4320838
10 ) Working on:  4349993
11 ) Working on:  4367956
12 ) Working on:  4372409
13 ) Working on:  4385724
14 ) Working on:  4392578
15 ) Working on:  4401084
16 ) Working on:  4416497
17 ) Working on:  4420741
18 ) Working on:  4422693
19 ) Working on:  4430479
20 ) Working on:  4432461
21 ) Working on:  4433174
22 ) Working on:  4433228
23 ) Working on:  4433323
24 ) Working on:  4433323
25 ) Working on:  4433720
26 ) Working on:  4433727
27 ) Working on:  4433827
28 ) Working on:  4434153
29 ) Working on:  4434501
30 ) Working on:  4434713
31 ) Working on:  4434788
32 ) Working on:  4434805
33 ) Working on:  4434928
34 ) Working on:  4435194
35 ) Working on:  4435427
36 ) Working on:  4435723
37 ) Working on:  4435852
38 ) Working on:  4435883
39 ) Working on:  443

In [189]:
print("Done!")

Done!


In [195]:
dataframes.shape, train.shape

((494, 105), (494, 105))

In [190]:
####without cross-promotion
train = pd.read_csv('links_cross_posting_info.csv')
features = [
                'consumptions_by_type__link_clicks',
                'consumptions_by_type__other_clicks', 
                'reactions_like_total',
                'stories_by_action_type__comment', 
                'stories_by_action_type__like',
                'stories_by_action_type__share', 
                'engaged_fan', 'fan_reach',
                'impressions', 
                'impressions_fan',
               ]
features = ['train_' + var for var in features]

####add the category variable for each of the pages in col_pages
# for pg in all_pages:
#     features.append(pg)

###overall clicks
res = ols(y = train['test_consumptions_by_type__link_clicks'], x = train[features])
print(res)

print('Share performance')

###24-hour share
#     res = ols(y = train['24_hours_share'], x = train[features])
#     print(res)

###overall share
res = ols(y = train['test_stories_by_action_type__share'], x = train[features])
print(res)


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <train_consumptions_by_type__link_clicks>
             + <train_consumptions_by_type__other_clicks> + <train_reactions_like_total>
             + <train_stories_by_action_type__comment> + <train_stories_by_action_type__like>
             + <train_stories_by_action_type__share> + <train_engaged_fan>
             + <train_fan_reach> + <train_impressions> + <train_impressions_fan>
             + <intercept>

Number of Observations:         423
Number of Degrees of Freedom:   11

R-squared:         0.9750
Adj R-squared:     0.9743

Rmse:           7634.3561

F-stat (10, 412):  1603.7332, p-value:     0.0000

Degrees of Freedom: model 10, resid 412

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
train_co

/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: The pandas.stats.ols module is deprecated and will be removed in a future version. We refer to external packages like statsmodels, see some examples here: http://www.statsmodels.org/stable/regression.html
  exec(code_obj, self.user_global_ns, self.user_ns)


In [196]:
####with cross-promotion
train = pd.read_csv('links_cross_posting_info.csv')

features = [
                'consumptions_by_type__link_clicks',
                'consumptions_by_type__other_clicks', 
                'reactions_like_total',
                'stories_by_action_type__comment', 
                'stories_by_action_type__like',
                'stories_by_action_type__share', 
                'engaged_fan', 'fan_reach',
                'impressions', 
                'impressions_fan',
               ]
features = ['train_' + var for var in features]

alt_inf = len(all_pages) + 1
train = train.replace(math.inf, alt_inf)

####add the category variable for each of the pages in col_pages
for pg in all_pages:
    features.append(pg)

###overall clicks
res = ols(y = train['test_consumptions_by_type__link_clicks'], x = train[features])
print(res)

print('Share performance')

###24-hour share
#     res = ols(y = train['24_hours_share'], x = train[features])
#     print(res)

###overall share
res = ols(y = train['test_stories_by_action_type__share'], x = train[features])
print(res)


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <train_consumptions_by_type__link_clicks>
             + <train_consumptions_by_type__other_clicks> + <train_reactions_like_total>
             + <train_stories_by_action_type__comment> + <train_stories_by_action_type__like>
             + <train_stories_by_action_type__share> + <train_engaged_fan>
             + <train_fan_reach> + <train_impressions> + <train_impressions_fan>
             + <BuzzFeed Books> + <BuzzFeed España> + <BuzzFeed Scotland> + <SOML>
             + <BuzzFeed News BR> + <Proper Tasty> + <BuzzFeed News>
             + <Obsessed by BuzzFeed> + <BuzzFeed France> + <BuzzFeed> + <BuzzFeed World>
             + <BuzzFeed Australia> + <LOLA> + <BuzzFeed Style> + <Top Knot>
             + <BuzzFeed México> + <BuzzFeed Weddings> + <BuzzFeed Entertainment>
             + <Cocoa Butter> + <Tasty Demais> + <BuzzFeed Open Lab> + <Bien Tasty>
             + <BuzzFeed Food> + <Buzz

/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: The pandas.stats.ols module is deprecated and will be removed in a future version. We refer to external packages like statsmodels, see some examples here: http://www.statsmodels.org/stable/regression.html
  exec(code_obj, self.user_global_ns, self.user_ns)


In [193]:
meta_df['buzz_id'].unique().shape

(26657,)

In [203]:
all_df.columns

Index(['external_id', 'stats_ts', 'stats_date',
       'consumptions_by_type__link_clicks',
       'consumptions_by_type__other_clicks', 'reactions_like_total',
       'stories_by_action_type__comment', 'stories_by_action_type__like',
       'stories_by_action_type__share', 'engaged_fan', 'fan_reach',
       'impressions', 'impressions_fan'],
      dtype='object')

In [222]:
meta_df.columns

Index(['external_id', 'parent_external_id', 'created_at', 'page', 'post_type',
       'buzz_id', 'uri', 'title'],
      dtype='object')

In [243]:
# import random
# sample_id = random.choice(meta_df['buzz_id'])
limit = 10
dataframes = pd.DataFrame()
count = 0
miss = 0

###Note that this is at a external_id level, and the predictions are on the post's performance within the page
for sample_id in meta_df['external_id'].unique()[:5000]:
    print(count,") Working on: ", sample_id)
    count += 1
    t_df = meta_df[meta_df['external_id'] == sample_id]
    output = t_df[['buzz_id', 'external_id', 'title', 'page']].drop_duplicates()
    output['start'] = t_df['created_at']
    
    all_pages = list(meta_df.page.unique())

    def get_cross_posted_pages(x):
        tf = meta_df[meta_df.buzz_id == x].sort_values('created_at', ascending = 1)
        return tf.page.unique()

    for buzz_id in output['buzz_id']:
        output['pages'] = output['buzz_id'].apply(get_cross_posted_pages)


    ##get earliest external_id for each buzz_id
#     def get_first_page(x):
#         tf = meta_df[meta_df.buzz_id == x].sort_values('created_at', ascending = 1)
#         return tf.external_id.values[0]

    output['pages'] = t_df['buzz_id'].apply(get_cross_posted_pages)
#     output['external_id'] = output['buzz_id'].apply(get_first_page)

    ###generate the order of cross-promotion:
    def generate_cat_pages(x, y):
        if(y in x):
            return list(x).index(y) + 1
        else:
            return math.inf

    for pag in all_pages:
    #     print("Getting for ", pag)
        output[pag] = output['pages'].apply(generate_cat_pages, y = pag)

    #####GET PERFORMANCE METRICS

    train_offset = 60
    test_offset = 24*60

    all_features = pd.DataFrame()
    all_predictions = pd.DataFrame()

    for external_id in output['external_id']:

        link_df = all_df[all_df['external_id'] == external_id]
        ###get the row for 1st 1 hour
        ###get the row for 1st day (24 hours)
        start = link_df['stats_date'].min()
        print(start)
        from datetime import datetime
        try:
            dt1 = datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
        except:
            continue
            
        ###within 30 minutes from posting
        import datetime
        ###set the initial offset period of interest - currently anything within the 1st 30 minutes!
        l_train_offset = datetime.timedelta(minutes=train_offset - 1)
        u_train_offset = datetime.timedelta(minutes=train_offset + 1)

        l_test_offset = datetime.timedelta(minutes=test_offset - 1)
        u_test_offset = datetime.timedelta(minutes=test_offset + 60)

        def get_timestamp(x):
            from datetime import datetime
            return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

        link_df['stats_timestamp'] = link_df['stats_date'].apply(get_timestamp)
        features = link_df[(link_df.stats_timestamp > dt1 + l_train_offset) & (link_df.stats_timestamp <= dt1 + u_train_offset)][:1]
        predictions = link_df[(link_df.stats_timestamp > dt1 + l_test_offset) & (link_df.stats_timestamp <= dt1 + u_test_offset)][:1]

        all_features = pd.concat([features,all_features])
        all_predictions = pd.concat([predictions,all_predictions])

        dependent_variables = ['external_id', 'consumptions_by_type__link_clicks', 'consumptions_by_type__other_clicks', 
                               'reactions_like_total', 'stories_by_action_type__comment', 'stories_by_action_type__like', 
                               'stories_by_action_type__share', 'engaged_fan', 'fan_reach',
                               'impressions', 'impressions_fan']
        predictor_variables = ['external_id', 'consumptions_by_type__link_clicks', 'stories_by_action_type__share']
        all_features = all_features[dependent_variables]
        all_predictions = all_predictions[predictor_variables]

        dependent_variables = ["train_" + var if var != "external_id" else var for var in dependent_variables]
        predictor_variables = ["test_" + var if var != "external_id" else var for var in predictor_variables]

        all_features.columns = dependent_variables
        all_predictions.columns = predictor_variables

        output = output.join(all_features.set_index('external_id'), on = 'external_id')
        output = output.join(all_predictions.set_index('external_id'), on = 'external_id')

        ###if there are NaNs, DROP - might need to change to handle certain Nans - replace with 0, for instance?
        if output.isnull().values.any():
            miss += 1
            continue
        dataframes = pd.concat([output, dataframes])
dataframes.to_csv('external_id_cross_posting_info.csv', index = False)

0 ) Working on:  322109601268040_1088954194583573
2017-02-27 03:35:04
1 ) Working on:  322109601268040_1130747220404270


/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2017-04-17 22:05:12
2 ) Working on:  322109601268040_1051507804994879
2017-01-08 02:05:04
3 ) Working on:  1433294486996353_1853196495006148
2017-04-04 21:00:54
4 ) Working on:  250557591819522_685088045033139
2017-05-13 07:10:07
5 ) Working on:  1433294486996353_1808633382795793
2017-01-21 19:04:13
6 ) Working on:  250557591819522_685084481700162
2017-05-12 15:05:07
7 ) Working on:  250557591819522_662096247332319
2017-03-27 14:05:06
8 ) Working on:  571035253040893_1090332724444474
2017-03-22 00:17:52
9 ) Working on:  160545781051273_247661595673024
2017-01-27 14:18:11
10 ) Working on:  322109601268040_1060100840802242
2017-01-19 03:35:05
11 ) Working on:  1433294486996353_1803371453321986
2017-01-13 03:00:46
12 ) Working on:  1737181656494507_1833377980208207
nan
13 ) Working on:  322109601268040_1088950301250629
2017-02-26 18:33:30
14 ) Working on:  618786471475708_1363519893669025
2017-01-02 00:35:04
15 ) Working on:  1433294486996353_1868221766836954
2017-04-29 18:05:12
16 ) Work

In [251]:
###regression

In [252]:
####without cross-promotion
train = pd.read_csv('external_id_cross_posting_info.csv')
features = [
                'consumptions_by_type__link_clicks',
                'consumptions_by_type__other_clicks', 
                'reactions_like_total',
                'stories_by_action_type__comment', 
                'stories_by_action_type__like',
                'stories_by_action_type__share', 
                'engaged_fan', 'fan_reach',
                'impressions', 
                'impressions_fan',
               ]
features = ['train_' + var for var in features]

####add the category variable for each of the pages in col_pages
# for pg in all_pages:
#     features.append(pg)

###overall clicks
res = ols(y = train['test_consumptions_by_type__link_clicks'], x = train[features])
print(res)

print('Share performance')

###24-hour share
#     res = ols(y = train['24_hours_share'], x = train[features])
#     print(res)

###overall share
res = ols(y = train['test_stories_by_action_type__share'], x = train[features])
print(res)


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <train_consumptions_by_type__link_clicks>
             + <train_consumptions_by_type__other_clicks> + <train_reactions_like_total>
             + <train_stories_by_action_type__comment> + <train_stories_by_action_type__like>
             + <train_stories_by_action_type__share> + <train_engaged_fan>
             + <train_fan_reach> + <train_impressions> + <train_impressions_fan>
             + <intercept>

Number of Observations:         84
Number of Degrees of Freedom:   11

R-squared:         0.9864
Adj R-squared:     0.9845

Rmse:          11389.5231

F-stat (10, 73):   528.5906, p-value:     0.0000

Degrees of Freedom: model 10, resid 73

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
train_consu

/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: The pandas.stats.ols module is deprecated and will be removed in a future version. We refer to external packages like statsmodels, see some examples here: http://www.statsmodels.org/stable/regression.html
  exec(code_obj, self.user_global_ns, self.user_ns)


In [253]:
####with cross-promotion
train = pd.read_csv('external_id_cross_posting_info.csv')

features = [
                'consumptions_by_type__link_clicks',
                'consumptions_by_type__other_clicks', 
                'reactions_like_total',
                'stories_by_action_type__comment', 
                'stories_by_action_type__like',
                'stories_by_action_type__share', 
                'engaged_fan', 'fan_reach',
                'impressions', 
                'impressions_fan',
               ]
features = ['train_' + var for var in features]

alt_inf = len(all_pages) + 1
train = train.replace(math.inf, alt_inf)

####add the category variable for each of the pages in col_pages
for pg in all_pages:
    features.append(pg)

###overall clicks
res = ols(y = train['test_consumptions_by_type__link_clicks'], x = train[features])
print(res)

print('Share performance')

###24-hour share
#     res = ols(y = train['24_hours_share'], x = train[features])
#     print(res)

###overall share
res = ols(y = train['test_stories_by_action_type__share'], x = train[features])
print(res)


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <train_consumptions_by_type__link_clicks>
             + <train_consumptions_by_type__other_clicks> + <train_reactions_like_total>
             + <train_stories_by_action_type__comment> + <train_stories_by_action_type__like>
             + <train_stories_by_action_type__share> + <train_engaged_fan>
             + <train_fan_reach> + <train_impressions> + <train_impressions_fan>
             + <BuzzFeed Books> + <BuzzFeed España> + <BuzzFeed Scotland> + <SOML>
             + <BuzzFeed News BR> + <Proper Tasty> + <BuzzFeed News>
             + <Obsessed by BuzzFeed> + <BuzzFeed France> + <BuzzFeed> + <BuzzFeed World>
             + <BuzzFeed Australia> + <LOLA> + <BuzzFeed Style> + <Top Knot>
             + <BuzzFeed México> + <BuzzFeed Weddings> + <BuzzFeed Entertainment>
             + <Cocoa Butter> + <Tasty Demais> + <BuzzFeed Open Lab> + <Bien Tasty>
             + <BuzzFeed Food> + <Buzz

/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: The pandas.stats.ols module is deprecated and will be removed in a future version. We refer to external packages like statsmodels, see some examples here: http://www.statsmodels.org/stable/regression.html
  exec(code_obj, self.user_global_ns, self.user_ns)
